In [1]:
import pandas as pd
import os

In [7]:
# Ottiene la directory di lavoro corrente
script_dir = os.getcwd()

# Costruisce il percorso relativo al file CSV
file_path = os.path.join(script_dir, "Data", "main_data", "languages.csv")

# Costruisce il percorso relativo al file CSV pulito
clean_path = os.path.join(script_dir, "Data", "clean_data")

# Crea la cartella clean_data se non esite
if not os.path.exists(clean_path):
    os.makedirs(clean_path)

# Carica il CSV in un DataFrame
df = pd.read_csv(file_path)

In [2]:
# Rimuove le virgolette
df['language'] = df['language'].str.replace('"', '', regex=False)

# Rimuove l'NBSP e sostituiscilo con uno spazio normale
df['language'] = df['language'].str.replace('\xa0', ' ', regex=False)

NameError: name 'df' is not defined

In [17]:
# Salva il file pulito
cleaned_csv_path = os.path.join(clean_path, "languages_cleaned.csv")
df.to_csv(cleaned_csv_path, index=False, encoding="utf-8")

Caricamento file csv pulito nel database con PostgreSQL


In [10]:
import psycopg2
from sqlalchemy import create_engine,text

In [11]:
#  Parametri di connessione PostgreSQL
DB_NAME = "ium_database"  
DB_USER = "postgres"       
DB_PASSWORD = "2004agnese"   
DB_HOST = "localhost"      
DB_PORT = "5432"    

conn = psycopg2.connect(dbname="postgres", user=DB_USER, password=DB_PASSWORD, host=DB_HOST, port=DB_PORT) # Crea una connessione al database usando psycopg2.connect
conn.autocommit = True # Conferma automaticamente ogni operazione evitando la conferma manuale dopo ogni query
cur = conn.cursor() # Crea un oggetto cur che permette di eseguire query SQL sul database
cur.execute(f"CREATE DATABASE {DB_NAME};") # Esegue il comando per creare il database con il nome specificato

DuplicateDatabase: ERRORE:  il database "ium_database" esiste già


In [12]:
# Crea un oggetto di connessione a un database PostgreSQL utilizzando SQLAlchemy
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [18]:
# Query SQL per creare la tabella
create_table_query = text("""
    CREATE TABLE IF NOT EXISTS languages_data (
        id INT,
        type TEXT,
        language TEXT
    );
""")

# Eseguire la query
with engine.connect() as connection: # Apre una connessione al database usando SQLAlchemy
    connection.execute(create_table_query) # Esegue il comando per la creazione della 
    connection.commit() # Salva le modifiche

In [19]:
# Legge il CSV pulito
df = pd.read_csv(cleaned_csv_path)

# Carica i dati nella tabella PostgreSQL
df.to_sql("languages_data", engine, if_exists="replace", index=False) # Esporta il dataframe nel database, sovrascrive e aggiunge i nuovi dati alla tabella esistente

762